In [88]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [89]:
df = pd.read_csv('data/train.csv')

In [90]:
df['Family'] = df['SibSp'] + df['Parch']

In [91]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin','SibSp','Parch'],inplace=True)

In [92]:
df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,0,3,male,22.0,7.2500,S,1
1,1,1,female,38.0,71.2833,C,1
2,1,3,female,26.0,7.9250,S,0
3,1,1,female,35.0,53.1000,S,1
4,0,3,male,35.0,8.0500,S,0


In [93]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
Embarked      2
Family        0
dtype: int64

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   Fare      891 non-null    float64
 5   Embarked  889 non-null    object 
 6   Family    891 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [95]:
# train test split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2)

In [96]:
# let's create pipeline to handle all things 

In [97]:
# for hadling numerical data,and scale only needed
numerical_features = ['Age','Fare']
numerical_transformer = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('scaling',StandardScaler())
])

# hadling categorical column ## here for Embarked feature we have to use oridinal encoding but , here purpose is understanding GridSearchCV
categorical_features = ['Embarked','Sex']
categorical_transformer = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='error',drop='first',sparse_output=False))
])

In [98]:
# putting in to column Transformer
preprocessor = ColumnTransformer([
    ('numric',numerical_transformer,numerical_features),
    ('categorical',categorical_transformer,categorical_features)
])

In [99]:
LR = Pipeline([
    ('preprocessor',preprocessor),
    ('model',LogisticRegression())
])

In [100]:
LR

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  ['Embarked', 'Sex'])])),
                ('model', LogisticRegression())])

In [101]:
# this is code for para grid
param_grid = {
    'preprocessor__numric__imputer__strategy': ['mean', 'median','mode'],
    'preprocessor__categorical__imputer__strategy': ['most_frequent', 'constant'],
    'model__C': [0.1, 1.0, 10, 100]
}
# keep in mind preprocesor->numric->imputer->strategy is flow how we define function
grid_search = GridSearchCV(LR, param_grid, cv=10)

In [102]:
grid_search.fit(X_train, y_train)

print(f"Best params:")
print(grid_search.best_params_)

Best params:
{'model__C': 0.1, 'preprocessor__categorical__imputer__strategy': 'most_frequent', 'preprocessor__numric__imputer__strategy': 'mean'}


In [103]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.779


In [106]:
# for all 10 cv score and all combination
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_model__C','param_preprocessor__categorical__imputer__strategy','param_preprocessor__numric__imputer__strategy','mean_test_score']]

,param_model__C,param_preprocessor__categorical__imputer__strategy,param_preprocessor__numric__imputer__strategy,mean_test_score
0,0.1,most_frequent,mean,0.779421
1,0.1,most_frequent,median,0.779421
3,0.1,constant,mean,0.779421
4,0.1,constant,median,0.779421
6,1.0,most_frequent,mean,0.778013
7,1.0,most_frequent,median,0.778013
9,1.0,constant,mean,0.778013
10,1.0,constant,median,0.778013
12,10,most_frequent,mean,0.778013
13,10,most_frequent,median,0.778013
